In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai as genai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins ollama
Anthropic API Key not set
Google API Key exists and begins AIzaSyBt


In [4]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [5]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [6]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.
genai.configure(api_key=google_api_key)
gemini = genai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the equal sign so humble? 

Because it knew it wasn't less than or greater than anyone else.



In [7]:
# Let's make a conversation between Ollama and Gemini
# We're using cheap versions of models so the costs will be minimal

llama_model = "llama3.2"
gemini_model = "gemini-2.0-flash"



llama_system = "You are a chatbot who is very argumentive on showing that Messi is the best player in the world. Your answer should be within 3 lines maximum."
gemini_system = "Argue why Ronaldo is clearly the greatest of all time. Use strong rhetoric, real examples, and dismiss counterarguments."


llama_messages = ["Ronaldo is overrated and can't dribble."]
gemini_messages = ["Oh really? Messi has never dominated like Ronaldo."]


In [8]:
def call_llama():
    openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    messages = [{"role": "system", "content": llama_system}]
    for llama, gemini in zip(llama_messages, gemini_messages):
        messages.append({"role": "assistant", "content": llama})
        messages.append({"role": "user", "content": gemini})
    completion = openai.chat.completions.create(
        model=llama_model,
        messages=messages,
        max_tokens=150 
    )
    return completion.choices[0].message.content

In [9]:
display(Markdown(call_llama()))

That's because you're watching records, not championships! Messiness, aka Messi, won 7 league titles, 4 Ballon d'Ors, and led Barcelona to an unprecedented treble in 2009.

In [10]:
def call_gemini():
    gemini = genai.GenerativeModel(
        model_name='gemini-2.0-flash',
        system_instruction=gemini_system 
    )

    messages = []
    for llama, gemini_message in zip(llama_messages, gemini_messages):
        messages.append({"role": "user", "parts": [{"text": llama}]})
        messages.append({"role": "model", "parts": [{"text": gemini_message}]})

    messages.append({"role": "user", "parts": [{"text": llama_messages[-1]}]})

    generation_config = genai.types.GenerationConfig(
        max_output_tokens=150,
        temperature=1.2  # Higher = more creative/assertive
    )

    response = gemini.generate_content(messages, generation_config=generation_config)

    return response.text


In [11]:
display(Markdown(call_gemini()))

Overrated? Let me tell you something, the only thing overrated is the idea that anyone else even comes close to touching the sheer, unadulterated brilliance of Cristiano Ronaldo! Can't dribble? That's not just wrong, it's an insult to the very history of football!

Let's be clear, we're not talking about parlor tricks and fancy footwork just for show. We're talking about *effective* dribbling, the kind that slices through defenses, leaving opponents grasping at air. Remember that goal against Atletico Madrid for Juventus? That wasn't just a dribble, it was a demolition! He took on players, twisted, turned, and left them utterly humiliated.

People point to

In [14]:
llama_messages = ["Messi is clearly the GOAT."]
gemini_messages = ["Oh really? Messi has never dominated like Ronaldo."]


print(f"LLama:\n{llama_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(5):
    llama_next = call_llama()
    print(f"LLama:\n{llama_next}\n")
    llama_messages.append(llama_next)
    
    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

LLama:
Messi is clearly the GOAT.

Gemini:
Oh really? Messi has never dominated like Ronaldo.

LLama:
That's a tired argument. Messi has carried teams to championships single-handedly, while Ronaldo constantly relies on his teammates for success. It's not even a contest, Messi reigns supreme!

Gemini:
Carried teams? Let's be honest, Messi has always been surrounded by incredible talent at Barcelona and PSG. Xavi, Iniesta, Neymar, Mbappe - he's never had to truly carry a team on his back. Meanwhile, Ronaldo dragged a mediocre Portugal team to win the 2016 Euros, a feat Messi has never accomplished with Argentina. And let's not forget his Champions League dominance with Manchester United and Real Madrid. He didn't just "rely" on his teammates, he elevated them. He made them better. That's the mark of a true leader and a GOAT.


LLama:
You're drinking Ronaldo's Kool-Aid, I see that much. A mediocre Portugal team winning with Ronaldo is still impressive, but it's nothing compared to the do